## 优化模型参数
现在我们有了一个模型和数据，是时候通过在数据上优化其参数来训练、验证和测试我们的模型了。  
训练一个模型是一个迭代的过程;在每次迭代(称为epoch)中，模型对输出进行预测，计算预测中的误差(损失)，收集误差对其参数的导数，并使用梯度下降优化这些参数。

In [2]:
# 数据+模型
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=False,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=False,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## 超参数
超参数是可调节的参数，可以让您控制模型优化过程，不同的超参数值会影响模型训练和收敛速度  

我们为训练定义了以下超参数:  
    Number of Epochs - 在数据集上迭代的次数  
    Batch Size       — 在参数更新之前通过网络传播的数据样本的数量  
    Learning Rate    - 在每个batch/epoch更新模型参数的多少。较小的值会导致学习速度较慢，而较大的值可能会导致训练过程中不可预测的行为。  

In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## 优化loop
设置了超参数之后，我们可以通过优化循环训练和优化我们的模型。一次优化的迭代成为一个epoch。  

每个epoch由两个主要部分组成：  
    1、训练loop：迭代训练数据集并尝试收敛到最优参数。  
    2、验证/测试loop：迭代测试数据集以检查模型性能是否正在提高。

## 损失函数
常用的损失函数包括均方误差nn.MSELoss用于回归任务;负对数似然nn.NLLLoss用于分类。nn.CrossEntropyLoss结合了nn.LogSoftmax和nn.NLLLoss。  
我们将模型的输出logit传递给nn.CrossEntropyLoss，将对数归一化并计算预测误差。

In [4]:
# 初始化损失函数
loss_fn = nn.CrossEntropyLoss()

## 优化器
优化是在每个训练步骤中调整模型参数以减少模型误差的过程。  
优化算法定义了这个过程是如何执行的(在这个例子中，我们使用随机梯度下降法)。所有优化逻辑都封装在优化器对象中。  
这里，我们使用SGD优化器。  

我们通过输入需要训练的模型参数，并传入学习率超参数来初始化优化器。

In [5]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

在训练loop中，优化分为三个步骤：  
1、调用optimizer.zero_grad()重置模型参数的梯度。默认情况下梯度是累积的，为了防止重复计算，我们在每次迭代时显式地将它们归零。  
2、通过调用loss.backward()反向传播预测损失。PyTorch保存每个参数的损耗w.r.t.的梯度。  
3、一旦我们有了梯度，我们调用optimizer.step()来通过向后传递中收集的梯度来调整参数。

## 完整实施
定义了对优化代码进行循环的train_loop，以及针对测试数据评估模型性能的test_loop。

In [12]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)  # 总大小
    for batch, (X,y) in enumerate(dataloader): # 迭代获取一个batch中的(X,y)以及batch的序号（0, 1, 2, 3, ...）
        #计算预测值和损失
        pred = model(X)
        loss = loss_fn(pred, y)  # 这里应该是在loss_fn里的交叉熵内自动取平均loss了

        #反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:  # 每100个batch遍历完数据集全集，一次epoch结束，输出100个batch的平均损失和当前准确度
            loss, current = loss.item(), batch * len(X)  
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)  # 总大小
    num_batches = len(dataloader) # batch的个数
    test_loss, correct =0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()  # 正确数

    test_loss /= num_batches # 平均一个iter的loss
    correct /= size  # 正确率
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [13]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.540704  [    0/60000]
loss: 0.637357  [ 6400/60000]
loss: 0.436373  [12800/60000]
loss: 0.679856  [19200/60000]
loss: 0.601338  [25600/60000]
loss: 0.579107  [32000/60000]
loss: 0.623697  [38400/60000]
loss: 0.659829  [44800/60000]
loss: 0.667456  [51200/60000]
loss: 0.608440  [57600/60000]
Test Error: 
 Accuracy: 79.1%, Avg loss: 0.600075 

Epoch 2
-------------------------------
loss: 0.528891  [    0/60000]
loss: 0.625786  [ 6400/60000]
loss: 0.427347  [12800/60000]
loss: 0.671320  [19200/60000]
loss: 0.594695  [25600/60000]
loss: 0.572922  [32000/60000]
loss: 0.612939  [38400/60000]
loss: 0.655929  [44800/60000]
loss: 0.662615  [51200/60000]
loss: 0.598744  [57600/60000]
Test Error: 
 Accuracy: 79.4%, Avg loss: 0.591761 

Epoch 3
-------------------------------
loss: 0.517989  [    0/60000]
loss: 0.615115  [ 6400/60000]
loss: 0.419075  [12800/60000]
loss: 0.663251  [19200/60000]
loss: 0.588324  [25600/60000]
loss: 0.567089  [32000/600